In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Loadimg the trained model
model = load_model('/content/drive/MyDrive/model.h5')

In [ ]:
# Load the preprocessed tweets
encoding = 'ISO-8859-1'
tweets_df = pd.read_csv('/content/drive/MyDrive/clean_tweets.csv',encoding=encoding)

In [ ]:
# Tokenizing same as the training dataset
max_features = 20000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(tweets_df['clean_text'].values)
tweet_tokens = tokenizer.texts_to_sequences(tweets_df['clean_text'].values)

# Pad the sequences to a fixed length consistent with the input shape of the model
tweet_tokens = pad_sequences(tweet_tokens, maxlen=39)

In [ ]:
# Make predictions using the trained model
predictions = model.predict(tweet_tokens)

302/302 [==============================] - 8s 18ms/step


In [ ]:
# Convert the categorical predictions to polarity value
#Drop neutral values
polarity = []
for i in predictions:
    if np.argmax(i) == -1:
        polarity.append(-1)
    elif np.argmax(i) == 1:
        polarity.append(1)

   

In [ ]:
# Add the predicted sentiments to the DataFrame
tweets_df['polarity'] = polarity
tweets_df.head(100)

,clean_text,polarity
0,my sim card does not have signal,-1
1,hi heisenberg we are sorry to hear this dm t...,-1
2,hi ive made an mpesa payment to the wrong ti...,-1
3,no phone numbers this time,-1
4,why are u ignoring uk,1
...,...,...
95,jambo willy apologies for the experience we ...,-1
96,number is conning people,-1
97,safaricom eats bundles like a starving victori...,1
98,haija safaricom post pay roll over function of...,1


In [ ]:
tweets_df['polarity'].value_counts()


-1    4978
 1    4677
Name: polarity, dtype: int64

In [ ]:
# Save the updated DataFrame to a new CSV file
tweets_df.to_csv('tweets_with_sentiments.csv', index=False)